In [2]:
pip install fastf1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.0/123.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 6.0 MB/s eta 0:00:00
  Created wheel for msgpack: filename=msgpack-1.0.2-cp312-cp312-linux_x86_64.whl size=15820 sha256=3b97bc37fa573ece5c45735944d0d94a4dfb19b106685fe88ef3ea2b06e2a754
  Stored in directory: /root/.cache/pip/wheels/67/a6/40/eda0983e595bbf3841af96dbff2340be72dfac96796fc3d578
Successfully built msgpack
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.1.2
    Uninstalling msgpack-1.1.2:
      Successfully uninstalled msgpack-1.1.2
  Attempting uninstall: websocket

In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
try:
    import xgboost as xgb
    HAS_XGB = True
except:
    HAS_XGB = False
    print("XGBoost not installed. Using RF and LR only.")

# 1. Load Data
print("Loading Data...")
train_data = pd.read_csv('f1_complete_dataset_sorted.csv')
test_data = pd.read_csv('f1_2025_test.csv')

# Features & Target
features = ['points', 'wins', 'podiums']
target = 'is_champion'

X_train = train_data[features]
y_train = train_data[target]

X_test = test_data[features]
y_test = test_data[target]

# 2. Train Models
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42)
}
if HAS_XGB:
    models["XGBoost"] = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

best_model = None
best_score = 0

print("\n Training Models & Testing on 2025 Data:")
print("-" * 40)

for name, model in models.items():
    # Train
    model.fit(X_train, y_train)

    # Predict on 2025 data
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)

    print(f"🔹 {name} Accuracy: {accuracy:.2f}")

📊 Loading Data...

🧠 Training Models & Testing on 2025 Data:
----------------------------------------
🔹 Random Forest Accuracy: 0.86
🔹 Logistic Regression Accuracy: 0.90
🔹 XGBoost Accuracy: 0.95


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [16:42:13] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [6]:
if accuracy > best_score:
        best_score = accuracy
        best_model = model

print("-" * 40)
print(f"Best Model Selected: {type(best_model).__name__}")

# 3. Manual Prediction Function
def predict_future_winner(points, wins, podiums):
    input_data = pd.DataFrame([[points, wins, podiums]], columns=features)
    prob = best_model.predict_proba(input_data)[0][1]
    is_winner = best_model.predict(input_data)[0]

    print(f"\n🔮 Prediction for [Pts: {points}, Wins: {wins}, Pods: {podiums}]:")

    if is_winner == 1:
        print(f"CHAMPION! (Confidence: {prob:.2%})")
    else:
        print(f"Not Champion (Confidence: {prob:.2%})")

# Example Manual Check
predict_future_winner(400, 15, 18) # Strong stats
predict_future_winner(200, 2, 5)   # Mid stats

----------------------------------------
Best Model Selected: XGBClassifier

🔮 Prediction for [Pts: 400, Wins: 15, Pods: 18]:
CHAMPION! (Confidence: 93.32%)

🔮 Prediction for [Pts: 200, Wins: 2, Pods: 5]:
Not Champion (Confidence: 0.18%)


In [7]:
# Features we are using (Strictly these 3)
features = ['points', 'wins', 'podiums']

# 1. Use the trained XGBoost model (Best Model)
best_model = models["XGBoost"]

# 2. Predict Probability instead of Yes/No
probs = best_model.predict_proba(X_test[features])[:, 1]

# 3. Add probability scores to our 2025 data
test_data['win_probability'] = probs

# 4. Find the driver with the MAXIMUM probability
winner_idx = test_data['win_probability'].idxmax()
predicted_winner = test_data.loc[winner_idx]

print("\n 2025 Championship Prediction (Based on Stats):")
print("--------------------------------------------------")
print(f"PREDICTED CHAMPION: {predicted_winner['driver'].upper()}")
print(f"   Team    : {predicted_winner['team']}")
print(f"   Points  : {predicted_winner['points']}")
print(f"   Wins    : {predicted_winner['wins']}")
print(f"   Podiums : {predicted_winner['podiums']}")
print(f"   Model Confidence: {predicted_winner['win_probability']:.2%}")
print("--------------------------------------------------")

# Let's verify Top 3 to see the fight
print("\n Top 3 Contenders in 2025:")
top_3 = test_data.sort_values(by='win_probability', ascending=False).head(3)
print(top_3[['driver', 'points', 'wins', 'podiums', 'win_probability']])


 2025 Championship Prediction (Based on Stats):
--------------------------------------------------
PREDICTED CHAMPION: LANDO NORRIS
   Team    : McLaren
   Points  : 423
   Wins    : 7
   Podiums : 18
   Model Confidence: 10.01%
--------------------------------------------------

 Top 3 Contenders in 2025:
           driver  points  wins  podiums  win_probability
0    Lando Norris     423     7       18         0.100144
1  Max Verstappen     421     8       15         0.100144
2   Oscar Piastri     410     7       16         0.100144


In [8]:
import joblib

joblib.dump(best_model, 'f1_champion_predictor.pkl')

print("Model Saved Successfully as 'f1_champion_predictor.pkl'")

Model Saved Successfully as 'f1_champion_predictor.pkl'
